In [1]:
import dspy

program = dspy.ChainOfThought('question -> anwser')
print(program.parameters)

e:\anaconda\envs\Test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<bound method BaseModule.parameters of self = Predict(StringSignature(question -> rationale, anwser
    instructions='Given the fields `question`, produce the fields `anwser`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the anwser}. We ...', '__dspy_field_type': 'output'})
    anwser = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Anwser:', 'desc': '${anwser}'})
))>


In [2]:
program.parameters

<bound method BaseModule.parameters of self = Predict(StringSignature(question -> rationale, anwser
    instructions='Given the fields `question`, produce the fields `anwser`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the anwser}. We ...', '__dspy_field_type': 'output'})
    anwser = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Anwser:', 'desc': '${anwser}'})
))>

In [3]:
from pydantic import BaseModel
class Human(BaseModel):
    name: str
    age: int
class MySig(dspy.Signature):
    text: str = dspy.InputField()
    question: str = dspy.InputField()
    peoples: list[Human]  = dspy.OutputField()

typed_cot = dspy.ChainOfThought(MySig)
typed_cot.parameters

<bound method BaseModule.parameters of self = Predict(StringSignature(text, question -> rationale, peoples
    instructions='Given the fields `text`, `question`, produce the fields `peoples`.'
    text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Text:', 'desc': '${text}'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the peoples}. We ...', '__dspy_field_type': 'output'})
    peoples = Field(annotation=list[Human] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Peoples:', 'desc': '${peoples}'})
))>

In [4]:
sample_text = 'Hello, my name is John and I am 25 years old. I am a software engineer.'
sample_question = 'extract the human information from the text'
train_set = [dspy.Example(sample_text=sample_text, question=sample_question, peoples=[Human(name='John', age=25)])]
def match(gold, predict, trace):
    return gold.peoples == predict.peoples
compiler = dspy.BootstrapFewShot(metric=match)
compiled_typed_cot = compiler.compile(typed_cot, trainset=train_set)

100%|██████████| 1/1 [00:00<00:00, 121.46it/s]


Bootstrapped 0 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.


In [5]:
compiled_typed_cot.save('compiled_typed_cot.json')

TypeError: Human(name='John', age=25) is not JSON serializable

In [7]:
compiled_typed_cot.demos

[Example({'sample_text': 'Hello, my name is John and I am 25 years old. I am a software engineer.', 'question': 'extract the human information from the text', 'peoples': [Human(name='John', age=25)]}) (input_keys=None)]